# Model Training

Model training step for the project '**Aftershock pattern prediction based on earthquake rupture data for improved seismic hazard assessment**' (pred_seism_aftXYZ). DeVries18 will refer to the article 'Deep learning of aftershock patterns following large earthquakes' by Phoebe M. R. DeVries, Fernanda Viégas, Martin Wattenberg & Brendan J. Meade, and published in Nature in 2018 (https://www.nature.com/articles/s41586-018-0438-y ).

Inputs from previous steps of the process model:
-  'Features_DeVries18.pkl': Features and target of baseline model (DeVries18), as pickle file;
-  'Features_new.pkl': New features and same target as baseline, as pickle file;
-  'model_baseline_DeVries18_init.h5': Baseline deep neural net of DeVries18, as (untrained) Keras model HDF5 file;
-  'model_baseline_DeVries18_simplified_init.h5': Simplified topology for baseline DNN of DeVries18, as (untrained) Keras model HDF5 file;
-  'model_DNN_init.h5': Deep neural net topology proposed for this project, as (untrained) Keras model HDF5 file;
-  'model_ANN_init.h5': Shallow artificial neural net proposed for this project, as (untrained) Keras model HDF5 file.

For comparison with the DeVries study, we will use the same training and test sets ('Training_FileNames.h5', 'Testing_FileNames.h5'), first imported from the Google Drive https://drive.google.com/drive/folders/1c5Rb_6EsuP2XedDjg37bFDyf8AadtGDa, as given in the README.md of https://github.com/phoebemrdevries/Learning-aftershock-location-patterns.

## Split dataset into training and testing sets

In [1]:
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IBM_API_KEY_ID': 'XXXXX',
    'IAM_SERVICE_ID': 'XXXXX',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.bluemix.net/oidc/token',
    'BUCKET': 'predseismaftxyz-donotdelete-pr-dfvzajzxij3spi',
    'FILE': 'Training_FileNames.h5'
}
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_2 = {
    'IBM_API_KEY_ID': 'XXXXX',
    'IAM_SERVICE_ID': 'XXXXX',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.bluemix.net/oidc/token',
    'BUCKET': 'predseismaftxyz-donotdelete-pr-dfvzajzxij3spi',
    'FILE': 'Testing_FileNames.h5'
}

from ibm_botocore.client import Config
import ibm_boto3

#Cloud Object Storage
cos1 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials_1['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials_1['ENDPOINT'])
cos2 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials_2['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials_2['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials_2['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials_2['ENDPOINT'])

In [2]:
#cos1.download_file(Bucket=credentials_1['BUCKET'], Key='Training_FileNames.h5', Filename='Training_FileNames.h5')
#cos2.download_file(Bucket=credentials_2['BUCKET'], Key='Testing_FileNames.h5', Filename='Testing_FileNames.h5')
!ls

Features_DeVries18.pkl			     model_DNN_init.h5
Features_new.pkl			     None0000000.png
LabelledDataset_DeVries18_balanced.pkl	     pred_seism_aftXYZ
LabelledDataset_DeVries18.pkl		     srcmod2.py
model_ANN_init.h5			     SRCMOD_cleaned.pkl
model_baseline_DeVries18_init.h5	     Testing_FileNames.h5
model_baseline_DeVries18_simplified_init.h5  Training_FileNames.h5


In [3]:
import h5py
import numpy as np
import pandas as pd
import sklearn

h5file1 = h5py.File('Training_FileNames.h5', 'r')
training_filenames = np.array(h5file1.get('file_names_training'))
h5file2 = h5py.File('Testing_FileNames.h5', 'r')
testing_filenames = np.array(h5file2.get('file_names_testing'))

training_IDs_temp = map(lambda x: str(x, 'utf-8'), training_filenames)   #from byte to string
training_IDs = list(map(lambda x: 's' + x[0:x.find('_')], training_IDs_temp))  #extract event tag only
testing_IDs_temp = map(lambda x: str(x, 'utf-8'), testing_filenames)   #from byte to string
testing_IDs = list(map(lambda x: 's' + x[0:x.find('_')], testing_IDs_temp))  #extract event tag only

In [4]:
Features_DeVries18 = pd.read_pickle('Features_DeVries18.pkl')
Features_new = pd.read_pickle('Features_new.pkl')

In [5]:
# create training & testing sets
TrainingSet_DeVries18 = Features_DeVries18.loc[Features_DeVries18['ID'].isin(training_IDs)]
TestingSet_DeVries18 = Features_DeVries18.loc[Features_DeVries18['ID'].isin(testing_IDs)]
TrainingSet_new = Features_new.loc[Features_new['ID'].isin(training_IDs)]
TestingSet_new = Features_new.loc[Features_new['ID'].isin(testing_IDs)]

# ratio of training samples
[len(TrainingSet_DeVries18), len(TestingSet_DeVries18), len(TrainingSet_DeVries18)/(len(TrainingSet_DeVries18)+len(TestingSet_DeVries18))]

[163912, 38732, 0.8088667811531552]

In [6]:
# shuffle training sets
TrainingSet_DeVries18 = sklearn.utils.shuffle(TrainingSet_DeVries18)
TrainingSet_new = sklearn.utils.shuffle(TrainingSet_new)

## Train baseline model

In [7]:
TrainingSet_DeVries18.head(10)

,ID,aftershocksyn,posabsxx,posabsxy,posabsyy,posabsxz,posabsyz,posabszz,negabsxx,negabsxy,negabsyy,negabsxz,negabsyz,negabszz
222405,s2004SUMATR01AMMO,0.0,0.267183,0.112649,0.188620,0.269481,0.204584,0.051750,-0.267183,-0.112649,-0.188620,-0.269481,-0.204584,-0.051750
6693,s1998IWATEJ01MIYA,0.0,0.000023,0.000051,0.000052,0.000017,0.000005,0.000005,-0.000023,-0.000051,-0.000052,-0.000017,-0.000005,-0.000005
2480,s2009LAQUIL05CIRE,0.0,0.006977,0.019410,0.022114,0.004669,0.006679,0.001286,-0.006977,-0.019410,-0.022114,-0.004669,-0.006679,-0.001286
13750,s2003TOKACH01TANI,0.0,0.047442,0.000795,0.001304,0.013542,0.000825,0.002027,-0.047442,-0.000795,-0.001304,-0.013542,-0.000825,-0.002027
23817,s2006KURILI01LAYx,1.0,0.047339,0.014700,0.016854,0.002011,0.002729,0.005463,-0.047339,-0.014700,-0.016854,-0.002011,-0.002729,-0.005463
10257,s2012SUMATR01HAYE,1.0,0.470913,0.507731,0.543225,0.217438,0.148457,0.048391,-0.470913,-0.507731,-0.543225,-0.217438,-0.148457,-0.048391
152793,s2004SUMATR01AMMO,0.0,0.783027,0.885536,0.946634,0.198831,0.303942,0.209710,-0.783027,-0.885536,-0.946634,-0.198831,-0.303942,-0.209710
10010,s1998IWATEJ01NAKA,0.0,0.000225,0.000321,0.000015,0.000956,0.000719,0.001658,-0.000225,-0.000321,-0.000015,-0.000956,-0.000719,-0.001658
8081,s2007SOLOMO01JIxx,0.0,0.006057,0.006502,0.011508,0.000932,0.000787,0.000154,-0.006057,-0.006502,-0.011508,-0.000932,-0.000787,-0.000154
104967,s2010MAULEC01HAYE,0.0,0.645532,0.187960,0.335242,0.854503,0.009194,0.092600,-0.645532,-0.187960,-0.335242,-0.854503,-0.009194,-0.092600


In [8]:
features = ['posabsxx', 'posabsxy', 'posabsyy', 'posabsxz', 'posabsyz', 'posabszz',
           'negabsxx', 'negabsxy', 'negabsyy', 'negabsxz', 'negabsyz', 'negabszz']
target = 'aftershocksyn'
x_train = TrainingSet_DeVries18[features]
y_train = TrainingSet_DeVries18[target]
x_test = TestingSet_DeVries18[features]
y_test = TestingSet_DeVries18[target]

# save for next step of the process model
x_test.to_pickle("TestingSet_X_DeVries18.pkl")
y_test.to_pickle("TestingSet_y_DeVries18.pkl")

In [9]:
import keras
from keras.models import load_model

# same as in DeVries18
baselinemodel_DeVries18 = load_model('model_baseline_DeVries18_init.h5')
batch_size = 3500
epochs = 5

history = baselinemodel_DeVries18.fit(x_train, y_train,
                            batch_size = batch_size,
                            epochs = epochs,
                            validation_split = 0.1)

Using TensorFlow backend.


Train on 147520 samples, validate on 16392 samples
Epoch 1/5
147520/147520 [==============================] - 4s 25us/step - loss: 0.6392 - binary_accuracy: 0.6663 - val_loss: 0.6024 - val_binary_accuracy: 0.7197
Epoch 2/5
147520/147520 [==============================] - 4s 24us/step - loss: 0.6069 - binary_accuracy: 0.7028 - val_loss: 0.6024 - val_binary_accuracy: 0.7199
Epoch 3/5
147520/147520 [==============================] - 4s 25us/step - loss: 0.5982 - binary_accuracy: 0.7118 - val_loss: 0.5908 - val_binary_accuracy: 0.7201
Epoch 4/5
147520/147520 [==============================] - 4s 24us/step - loss: 0.5944 - binary_accuracy: 0.7158 - val_loss: 0.5991 - val_binary_accuracy: 0.7202
Epoch 5/5
147520/147520 [==============================] - 4s 26us/step - loss: 0.5913 - binary_accuracy: 0.7181 - val_loss: 0.5941 - val_binary_accuracy: 0.7202


This is the baseline model CV results - we should aim at achieving similar or greater accuracy ($\geq$ 72%) for the new model with new set of features. This will however not assure a same or better generalization than the baseline model (AUC = 85%, see next step of the process model).

In [10]:
baselinemodel_DeVries18.save('model_baseline_DeVries18_trained.h5')

### Side note

Since the Devries18 model uses a relatively small input layer (12 nodes), a simpler DNN topology should do just fine. We show it, only for illustrative purpose. The model proposed in this project will use another set of features.

In [11]:
baselinemodel_DeVries18_simplified = load_model('model_baseline_DeVries18_simplified_init.h5')

history = baselinemodel_DeVries18_simplified.fit(x_train, y_train,
                            batch_size = batch_size,
                            epochs = epochs,
                            validation_split = 0.1)

Train on 147520 samples, validate on 16392 samples
Epoch 1/5
147520/147520 [==============================] - 0s 3us/step - loss: 0.6287 - binary_accuracy: 0.6972 - val_loss: 0.5995 - val_binary_accuracy: 0.7177
Epoch 2/5
147520/147520 [==============================] - 0s 2us/step - loss: 0.5984 - binary_accuracy: 0.7109 - val_loss: 0.5844 - val_binary_accuracy: 0.7147
Epoch 3/5
147520/147520 [==============================] - 0s 2us/step - loss: 0.5906 - binary_accuracy: 0.7115 - val_loss: 0.5805 - val_binary_accuracy: 0.7157
Epoch 4/5
147520/147520 [==============================] - 0s 2us/step - loss: 0.5876 - binary_accuracy: 0.7130 - val_loss: 0.5782 - val_binary_accuracy: 0.7165
Epoch 5/5
147520/147520 [==============================] - 0s 2us/step - loss: 0.5859 - binary_accuracy: 0.7142 - val_loss: 0.5767 - val_binary_accuracy: 0.7172


In [12]:
# note the similar CV accuracy using 12 - 8 - 8 - 1 instead of 12 - 50 - 50 - 50 - 50 - 50 - 50 - 1
baselinemodel_DeVries18_simplified.save('model_baseline_DeVries18_simplified_trained.h5')

## Train new models

Training new models with the proposed set of features based on geometry and kinematics (minimum distance to mainshock rupture and mean slip on rupture).

In [13]:
#features = ['mindist', 'dipMean', 'strikeMean', 'slipMean']   #best 4 - 12 - 12 - 1
features = ['mindist', 'slipMean']   #best 2 - 6 - 6 - 1
target = 'aftershocksyn'
x_train = TrainingSet_new[features]
y_train = TrainingSet_new[target]
x_test = TestingSet_new[features]
y_test = TestingSet_new[target]

# save for next step of the process model
x_test.to_pickle("TestingSet_X_new.pkl")
y_test.to_pickle("TestingSet_y_new.pkl")

### Train new DNN (simplified topology)

In [14]:
model_DNN = load_model('model_DNN_init.h5')

history = model_DNN.fit(x_train, y_train,
                            batch_size = batch_size,
                            epochs = epochs,
                            validation_split = 0.1)

Train on 147520 samples, validate on 16392 samples
Epoch 1/5
147520/147520 [==============================] - 0s 3us/step - loss: 0.6853 - binary_accuracy: 0.6374 - val_loss: 0.6728 - val_binary_accuracy: 0.7247
Epoch 2/5
147520/147520 [==============================] - 0s 1us/step - loss: 0.6650 - binary_accuracy: 0.6413 - val_loss: 0.6426 - val_binary_accuracy: 0.7227
Epoch 3/5
147520/147520 [==============================] - 0s 1us/step - loss: 0.6359 - binary_accuracy: 0.6477 - val_loss: 0.6053 - val_binary_accuracy: 0.7277
Epoch 4/5
147520/147520 [==============================] - 0s 1us/step - loss: 0.6171 - binary_accuracy: 0.6596 - val_loss: 0.5869 - val_binary_accuracy: 0.7343
Epoch 5/5
147520/147520 [==============================] - 0s 1us/step - loss: 0.6078 - binary_accuracy: 0.6676 - val_loss: 0.5780 - val_binary_accuracy: 0.7366


In [ ]:
# Documenting a first DNN model result
# we tried the following topology 4 - 12 - 12 - 1, the 4 features being: ['mindist', 'dipMean', 'strikeMean', 'slipMean']
# we obtained: 
#Epoch 5/5
#147801/147801 [==============================] - 0s 2us/step - loss: 0.6134 - binary_accuracy: 0.6829 - 
#                                                           val_loss: 0.5801 - val_binary_accuracy: 0.7228

# additional tuning involved changes in topology, using relu instead of tanh activation, dropout rate change [NOT SHOWN for clarity].

In [15]:
model_DNN.save('model_DNN_trained.h5')

### Train ANN

Training of an Artificial Neural Network with one hidden layer (no deep learning!).

In [16]:
model_ANN = load_model('model_ANN_init.h5')

history = model_ANN.fit(x_train, y_train,
                            batch_size = batch_size,
                            epochs = epochs,
                            validation_split = 0.1)

Train on 147520 samples, validate on 16392 samples
Epoch 1/5
147520/147520 [==============================] - 0s 3us/step - loss: 0.6547 - binary_accuracy: 0.7135 - val_loss: 0.6359 - val_binary_accuracy: 0.7220
Epoch 2/5
147520/147520 [==============================] - 0s 1us/step - loss: 0.6212 - binary_accuracy: 0.7232 - val_loss: 0.6095 - val_binary_accuracy: 0.7216
Epoch 3/5
147520/147520 [==============================] - 0s 1us/step - loss: 0.5987 - binary_accuracy: 0.7274 - val_loss: 0.5917 - val_binary_accuracy: 0.7296
Epoch 4/5
147520/147520 [==============================] - 0s 1us/step - loss: 0.5836 - binary_accuracy: 0.7339 - val_loss: 0.5797 - val_binary_accuracy: 0.7354
Epoch 5/5
147520/147520 [==============================] - 0s 1us/step - loss: 0.5735 - binary_accuracy: 0.7367 - val_loss: 0.5718 - val_binary_accuracy: 0.7371


In [17]:
model_ANN.save('model_ANN_trained.h5')

### Train XGBoost classifier

To not only test neural networks, we here test another standard machine learning algorithm. XGBoost has become a method of choice in recent Kaggle competitions (althought this does not mean that it will perform better than neural networks in the present case - "No Free Lunch" theorem):

In [18]:
import xgboost as xgb
from sklearn.grid_search import GridSearchCV

params = {
    'max_depth': [5, 7, 10]
}
gridsearch = GridSearchCV(estimator = xgb.XGBClassifier(
                          objective = "binary:logistic"
                          ),
                        param_grid = params,
                        scoring='accuracy',
                        n_jobs=1,
                        iid=False,
                        cv=5)
gridsearch.fit(x_train, y_train)
gridsearch.grid_scores_, gridsearch.best_params_, gridsearch.best_score_

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


([mean: 0.78262, std: 0.00259, params: {'max_depth': 5},
  mean: 0.78558, std: 0.00254, params: {'max_depth': 7},
  mean: 0.78534, std: 0.00197, params: {'max_depth': 10}],
 {'max_depth': 7},
 0.7855800447128167)

In [19]:
#!pip install joblib

    100% |████████████████████████████████| 286kB 3.5MB/s eta 0:00:01


In [20]:
from joblib import dump

dump(gridsearch, 'model_XGBoost.joblib') 
!ls

Features_DeVries18.pkl				model_XGBoost.joblib
Features_new.pkl				None0000000.png
LabelledDataset_DeVries18_balanced.pkl		pred_seism_aftXYZ
LabelledDataset_DeVries18.pkl			srcmod2.py
model_ANN_init.h5				SRCMOD_cleaned.pkl
model_ANN_trained.h5				Testing_FileNames.h5
model_baseline_DeVries18_init.h5		TestingSet_X_DeVries18.pkl
model_baseline_DeVries18_simplified_init.h5	TestingSet_X_new.pkl
model_baseline_DeVries18_simplified_trained.h5	TestingSet_y_DeVries18.pkl
model_baseline_DeVries18_trained.h5		TestingSet_y_new.pkl
model_DNN_init.h5				Training_FileNames.h5
model_DNN_trained.h5
